In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
import torch


from qibo import Circuit, gates, set_backend
from qibo.symbols import Z
from qibo.hamiltonians import SymbolicHamiltonian

from qiboml.models.encoding import PhaseEncoding, BinaryEncoding
from qiboml.models.decoding import Expectation
from qiboml.operations.differentiation import PSR

2025-06-10 09:20:18.678668: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


First of all, let's create an encoding, a decoding and a trainable circuit.

In [2]:
nqubits = 2
encoding = BinaryEncoding(nqubits=nqubits)

# we construct a trainable parametrized circuit
# that is the core of our quantum model
circuit = Circuit(nqubits)
for _ in range(5):
    for q in range(nqubits):
        circuit.add(gates.RY(q, theta=np.random.randn() * np.pi))
        circuit.add(gates.RZ(q, theta=np.random.randn() * np.pi))
    circuit.add(gates.CNOT(0,1))

observable = SymbolicHamiltonian(Z(0), nqubits=nqubits)
# and then construct the expectation decoder
decoding = Expectation(nqubits=nqubits, observable=observable)

[Qibo 0.2.18|INFO|2025-06-10 09:20:29]: Using numpy backend on /CPU:0


## Keras

The initialisation procedure allows three different scenarios: 

(1) No initialisation from the user, which triggers the default settings (gaussian init of the parameters); \
(2) Initialisation with a np.ndarray; \
(3) Initialisation with tf.keras.initializers.Initializer; 

In [3]:
from qiboml.interfaces.keras import QuantumModel
set_backend(backend="qiboml", platform="tensorflow")

[Qibo 0.2.18|INFO|2025-06-10 09:11:53]: Using qiboml (tensorflow) backend on /device:CPU:0


Let's test the first scenario: no initialisation.

In [4]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    decoding=decoding
)
q_model.circuit_parameters

<Variable path=quantum_model/variable, shape=(20,), dtype=float32, value=[-0.00348381  0.00055974  0.00169785  0.00889099 -0.02072026  0.00905712
 -0.00488633  0.01032944 -0.00485224 -0.00010711  0.00524021  0.0123707
  0.00753574 -0.01869221  0.00583563  0.00674955 -0.00218423 -0.00207006
  0.00903391 -0.00558459]>

Now, let's test the second scenario scenario: np.ndarray. 
In this case we must be careful with the shape of the array: as an example I will show what happens when the shape is not correct and when the shape is correct;

In [5]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    angles_initialisation = np.ones(shape=(100, 100)),
    decoding=decoding
)

ValueError: Shape not valid for angles_initialisation. The shape should be (20,).

In [ ]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    angles_initialisation = np.ones(shape=(20,)),
    decoding=decoding
)
q_model.circuit_parameters

<Variable path=quantum_model_2/variable_1, shape=(20,), dtype=float32, value=[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]>

Finally, we show the last possibility for parameters initialisation: tf.keras.initializers.Initializer.
This technique is convenient because it does not require to be careful with the shape of the parameters.

In [7]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    angles_initialisation = keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),
    decoding=decoding
)
q_model.circuit_parameters

<Variable path=quantum_model_3/variable_2, shape=(20,), dtype=float32, value=[-0.02237872 -0.03726705  0.02841783  0.04559671 -0.02656081  0.04821536
 -0.01313218  0.00604746 -0.0369916  -0.04520694  0.00813205 -0.00796653
  0.00999292 -0.00953228  0.03554766  0.03193274 -0.01563294  0.0425458
 -0.02057029  0.0278796 ]>

## PyTorch

The initialisation procedure allows three different scenarios: 

(1) No initialisation from the user, which triggers the default settings (gaussian init of the parameters); \
(2) Initialisation with a np.ndarray; \
(3) Initialisation with torch.nn.init; 

In [3]:
from qiboml.interfaces.pytorch import QuantumModel
set_backend(backend="qiboml", platform="pytorch")

[Qibo 0.2.18|INFO|2025-06-10 09:20:35]: Using qiboml (pytorch) backend on cpu


In [4]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    decoding=decoding
)
q_model.circuit_parameters

Parameter containing:
tensor([ 2.0815e-03,  6.3532e-03, -2.1408e-02, -1.2228e-02, -1.6009e-03,
         4.8261e-03,  7.8765e-03,  1.5450e-02,  5.2157e-03, -3.8650e-03,
         1.1582e-02,  1.1548e-03, -1.7946e-02,  5.9990e-03,  7.5558e-03,
         4.8045e-03, -7.1005e-05,  2.0482e-03,  6.0442e-03,  1.2766e-02],
       requires_grad=True)

Now, let's test the second scenario scenario: np.ndarray. 
In this case we must be careful with the shape of the array: as an example I will show what happens when the shape is not correct and when the shape is correct;

In [5]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    angles_initialisation=np.ones(shape=(100, 100)),
    decoding=decoding
)
q_model.circuit_parameters

ValueError: Shape not valid for angles_initialisation. The shape should be (20,).

In [7]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    angles_initialisation=np.ones(shape=(20,)),
    decoding=decoding
)
q_model.circuit_parameters

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], dtype=torch.float64, requires_grad=True)

In [6]:
q_model = QuantumModel(
    circuit_structure=[encoding, circuit],
    angles_initialisation=torch.nn.init.uniform_,
    decoding=decoding
)
q_model.circuit_parameters

tensor([0.4116, 0.9851, 0.8940, 0.3274, 0.2381, 0.7679, 0.4386, 0.3742, 0.6395,
        0.2862, 0.0769, 0.6143, 0.0378, 0.2268, 0.1345, 0.6334, 0.8914, 0.7569,
        0.7454, 0.1061])